***@Author: Ranjith G C**
<br>
***@Date: 2021-09-29***
<br>
***@Last Modified by: Ranjith G C***
<br>
***@Last Modified time: 2021-09-29***
<br>
***@Title : Program Aim to practice on facebook dataset using spark sql operations.***

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName('facebook_data').getOrCreate()
sc=spark.sparkContext
from pyspark.sql import *
from pyspark.sql.functions import col, max as max_, min as min_, unix_timestamp
from loghandler import logger

In [2]:
df = spark.read.format("csv").option("header", "true").load("hdfs://localhost:9000/datasets/*.csv")
df.createOrReplaceTempView("ranjith")

# Finding The Total Number Of Users in This Datasets

In [14]:
total_users = spark.sql("select count(*) as total_user from ranjith")
total_users.show()

+----------+
|total_user|
+----------+
|     99003|
+----------+



# Finding The Total Number Of Facebook User Whose is Above 25

In [17]:
users_above_25_years = spark.sql("select count(*) as user_above_25_years from ranjith where age > 25")
users_above_25_years.show()

+-------------------+
|user_above_25_years|
+-------------------+
|              56676|
+-------------------+



# Finding Which User Have More Friends Male or Female.

In [16]:
more_friends = spark.sql("select gender,avg(friend_count) as friend_count from ranjith group by gender")
more_friends.show()

+------+------------------+
|gender|      friend_count|
+------+------------------+
|    NA|184.41142857142856|
|female|241.96994087544095|
|  male|165.03545941885477|
+------+------------------+



# Who Received More Likes Between Young and Old People

In [18]:
young_people = spark.sql("select avg(likes_received) as young_people_likes from ranjith where age>=13 and age <= 25")
old_people = spark.sql("select avg(likes_received) as old_people_likes from ranjith where age >= 35")
young_people.show()
old_people.show()

+------------------+
|young_people_likes|
+------------------+
| 200.2870508186264|
+------------------+

+------------------+
|  old_people_likes|
+------------------+
|103.89021217994491|
+------------------+



# Finding Out The Count Of Facebook User For Each BirthDay Month

In [19]:
user_birthday_month_count = spark.sql("select dob_month,count(*) as birthday_month_count from ranjith group by dob_month")
user_birthday_month_count.show()

+---------+--------------------+
|dob_month|birthday_month_count|
+---------+--------------------+
|        7|                8021|
|       11|                7205|
|        3|                8110|
|        8|                8266|
|        5|                8271|
|        6|                7607|
|        9|                7939|
|        1|               11772|
|       10|                8476|
|        4|                7810|
|       12|                7894|
|        2|                7632|
+---------+--------------------+



# Do Young People Use Mobile Or Computer For Browsing Facebook and Old People use mobile or computer for browsing facebook

In [20]:
print("************** Young Users *****************")
young_people_choice = spark.sql("select avg(mobile_likes_received) as mobile_uses,avg(www_likes_received) as computer_uses from ranjith where age >= 13 and age <= 25")
old_people_choice = spark.sql("select avg(mobile_likes_received) as mobile_uses,avg(www_likes_received) as computer_uses from ranjith where age>= 35")
young_people_choice.show()
print("************** Old Users *****************")
old_people_choice.show()

************** Young Users *****************
+-----------------+-----------------+
|      mobile_uses|    computer_uses|
+-----------------+-----------------+
|119.8740283979493|80.41295154393177|
+-----------------+-----------------+

************** Old Users *****************
+-----------------+------------------+
|      mobile_uses|     computer_uses|
+-----------------+------------------+
|57.34956135876772|46.540625318779966|
+-----------------+------------------+



# Saving facebook data into hive table

In [4]:
spark = SparkSession.builder.master("local").appName('facebook_data').enableHiveSupport().getOrCreate()

In [5]:
fb_data2 = df.select("userid","age","dob_day","dob_year","dob_month","gender","tenure","friend_count","friendships_initiated","likes","likes_received","mobile_likes","mobile_likes_received","www_likes","www_likes_received")

In [7]:
fb_data2.write.mode("overwrite").saveAsTable("default.facebook_data2")

In [9]:
spark.sql("select *from default.facebook_data2").show()

+-------+---+-------+--------+---------+------+------+------------+---------------------+-----+--------------+------------+---------------------+---------+------------------+
| userid|age|dob_day|dob_year|dob_month|gender|tenure|friend_count|friendships_initiated|likes|likes_received|mobile_likes|mobile_likes_received|www_likes|www_likes_received|
+-------+---+-------+--------+---------+------+------+------------+---------------------+-----+--------------+------------+---------------------+---------+------------------+
|2094382| 14|     19|    1999|       11|  male|   266|           0|                    0|    0|             0|           0|                    0|        0|                 0|
|1192601| 14|      2|    1999|       11|female|     6|           0|                    0|    0|             0|           0|                    0|        0|                 0|
|2083884| 14|     16|    1999|       11|  male|    13|           0|                    0|    0|             0|           0|  